---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

# Assignment 3 - Evaluation

In this assignment you will train several models and evaluate how effectively they predict instances of fraud using data based on [this dataset from Kaggle](https://www.kaggle.com/dalpozz/creditcardfraud).
 
Each row in `fraud_data.csv` corresponds to a credit card transaction. Features include confidential variables `V1` through `V28` as well as `Amount` which is the amount of the transaction. 
 
The target is stored in the `class` column, where a value of 1 corresponds to an instance of fraud and 0 corresponds to an instance of not fraud.

In [101]:
import numpy as np
import pandas as pd

### Question 1
Import the data from `fraud_data.csv`. What percentage of the observations in the dataset are instances of fraud?

*This function should return a float between 0 and 1.* 

In [102]:
def answer_one():
    
    # Your code here
    
    df = pd.read_csv('fraud_data.csv')
    count_fraud = df['Class'].sum(axis = 0)
    fraud_rate = count_fraud / df.shape[0]
    
    return fraud_rate# Return your answer

answer_one()

0.016410823768035772

In [103]:
# Use X_train, X_test, y_train, y_test for all of the following questions
from sklearn.model_selection import train_test_split

df = pd.read_csv('fraud_data.csv')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Question 2

Using `X_train`, `X_test`, `y_train`, and `y_test` (as defined above), train a dummy classifier that classifies everything as the majority class of the training data. What is the accuracy of this classifier? What is the recall?

*This function should a return a tuple with two floats, i.e. `(accuracy score, recall score)`.*

In [104]:
def answer_two():
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import accuracy_score, recall_score
    
    # Your code here
    
    dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    y_dummy_predictions = dummy_majority.predict(X_test)
    accu = accuracy_score(y_test, y_dummy_predictions)
    rec = recall_score(y_test, y_dummy_predictions)
    
    return accu, rec# Return your answer

answer_two()

(0.98525073746312686, 0.0)

### Question 3

Using X_train, X_test, y_train, y_test (as defined above), train a SVC classifer using the default parameters. What is the accuracy, recall, and precision of this classifier?

*This function should a return a tuple with three floats, i.e. `(accuracy score, recall score, precision score)`.*

In [38]:
def answer_three():
    from sklearn.metrics import accuracy_score, recall_score, precision_score
    from sklearn.svm import SVC

    # Your code here
    
    svm = SVC().fit(X_train, y_train)
    y_svm_predicted = svm.predict(X_test)
    accu = accuracy_score(y_test, y_svm_predicted)
    rec = recall_score(y_test, y_svm_predicted)
    pre = precision_score(y_test, y_svm_predicted)
    
    return accu, rec, pre# Return your answer

answer_three()

(0.99078171091445433, 0.375, 1.0)

### Question 4

Using the SVC classifier with parameters `{'C': 1e9, 'gamma': 1e-07}`, what is the confusion matrix when using a threshold of -220 on the decision function. Use X_test and y_test.

*This function should return a confusion matrix, a 2x2 numpy array with 4 integers.*

In [45]:
def answer_four():
    from sklearn.metrics import confusion_matrix
    from sklearn.svm import SVC

    # Your code here
    
    svm = SVC(C = 1e9, gamma = 1e-07).fit(X_train, y_train)
    y_scores = svm.decision_function(X_test)
    threshold = -220
    y_svm_predicted = (y_scores > threshold)
#     y_svm_predicted = svm.predict(X_test)
    confusion = confusion_matrix(y_test, y_svm_predicted)
    
    return confusion# Return your answer

answer_four()

array([[5320,   24],
       [  14,   66]])

### Question 5

Train a logisitic regression classifier with default parameters using X_train and y_train.

For the logisitic regression classifier, create a precision recall curve and a roc curve using y_test and the probability estimates for X_test (probability it is fraud).

Looking at the precision recall curve, what is the recall when the precision is `0.75`?

Looking at the roc curve, what is the true positive rate when the false positive rate is `0.16`?

*This function should return a tuple with two floats, i.e. `(recall, true positive rate)`.*

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

def answer_five():
        
    # Your code here
    
    lr = LogisticRegression().fit(X_train, y_train)
    y_scores_lr = lr.decision_function(X_test)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores_lr)
    ind = [i for i, prec in enumerate(precision) if prec == 0.75]
    rec = recall[ind[0]]

    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_scores_lr)
    inde = [i for i, fpr in enumerate(fpr_lr) if np.abs(fpr-0.16)<0.0005]
    tpr = (tpr_lr[inde[0]] + tpr_lr[inde[1]]) / 2
    
    y_proba_lr = lr.predict_proba(X_test)
    
    return rec, tpr# Return your answer

answer_five()

[[  9.98749097e-01   1.25090280e-03]
 [  9.98542155e-01   1.45784455e-03]
 [  9.97504691e-01   2.49530907e-03]
 ..., 
 [  9.97862537e-01   2.13746337e-03]
 [  9.99587072e-01   4.12928407e-04]
 [  9.98022349e-01   1.97765092e-03]]


(0.82499999999999996, 0.94374999999999998)

### Question 6

Perform a grid search over the parameters listed below for a Logisitic Regression classifier, using recall for scoring and the default 3-fold cross validation.

`'penalty': ['l1', 'l2']`

`'C':[0.01, 0.1, 1, 10, 100]`

From `.cv_results_`, create an array of the mean test scores of each parameter combination. i.e.

|      	| `l1` 	| `l2` 	|
|:----:	|----	|----	|
| **`0.01`** 	|    ?	|   ? 	|
| **`0.1`**  	|    ?	|   ? 	|
| **`1`**    	|    ?	|   ? 	|
| **`10`**   	|    ?	|   ? 	|
| **`100`**   	|    ?	|   ? 	|

<br>

*This function should return a 5 by 2 numpy array with 10 floats.* 

*Note: do not return a DataFrame, just the values denoted by '?' above in a numpy array. You might need to reshape your raw result to meet the format we are looking for.*

In [100]:
def answer_six():    
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    # Your code here
    
    lr = LogisticRegression()
    grid_values = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100]}

    grid_lr_rec = GridSearchCV(lr, param_grid = grid_values, scoring = 'recall', cv = 3)
    grid_lr_rec.fit(X_train, y_train)
    result = grid_lr_rec.cv_results_['mean_test_score'].reshape(5,2)
    
    return result# Return your answer

answer_six()

array([[ 0.66666667,  0.76086957],
       [ 0.80072464,  0.80434783],
       [ 0.8115942 ,  0.8115942 ],
       [ 0.80797101,  0.8115942 ],
       [ 0.80797101,  0.80797101]])

In [ ]:
# Use the following function to help visualize results from the grid search
def GridSearch_Heatmap(scores):
    %matplotlib notebook
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure()
    sns.heatmap(scores.reshape(5,2), xticklabels=['l1','l2'], yticklabels=[0.01, 0.1, 1, 10, 100])
    plt.yticks(rotation=0);

#GridSearch_Heatmap(answer_six())